In [1]:
import yfinance as yf
import pandas as pd
import psycopg2
import configparser
from datetime import datetime
import nbimporter
from General import *
from Functions import *
from SQL import *
from Connection_to_pgAdmin4 import *
from AWS_PostgreSQL import*

In [2]:
'''

The Python script is designed to automate the process of updating a financial database with the latest stock information.
It begins by connecting to a database to retrieve a list of stock symbols and their last recorded data dates. 
For each symbol, it fetches new stock data from Yahoo Finance, starting from the day after the last recorded date until the current date. 
This new data includes various financial metrics such as opening price, highest price, lowest price, closing price, volume, dividends, and stock splits. 
Once the data is fetched, it is inserted into the source.historical_stock_data table in the database. 
The script uses custom functions from imported modules for database connections, data fetching, and SQL operations.
If executed as the main program, the script will carry out the update process from start to finish, provided all the necessary modules and functions are correctly defined and available.
The script includes a user warning about the potential for large data transfers and database modifications.

'''

'\n\nThe Python script is designed to automate the process of updating a financial database with the latest stock information.\nIt begins by connecting to a database to retrieve a list of stock symbols and their last recorded data dates. \nFor each symbol, it fetches new stock data from Yahoo Finance, starting from the day after the last recorded date until the current date. \nThis new data includes various financial metrics such as opening price, highest price, lowest price, closing price, volume, dividends, and stock splits. \nOnce the data is fetched, it is inserted into the source.historical_stock_data table in the database. \nThe script uses custom functions from imported modules for database connections, data fetching, and SQL operations.\nIf executed as the main program, the script will carry out the update process from start to finish, provided all the necessary modules and functions are correctly defined and available.\nThe script includes a user warning about the potential fo

In [11]:
#*****************
#This function reads the data from the facts.dates_of_stock_info
#Repeatable
#*****************

#Reference cell D Name
def fetch_symbols_and_last_dates_AWS_postgreSQL():
    connection = connect_to_database_AWS_postgreSQL()
    if connection:
        try:
            query = """
            SELECT symbol, lastdate
            FROM f_dates_of_stock_info;
            """
            symbols_dates_df = pd.read_sql(query, connection)
            return symbols_dates_df
        except (Exception, psycopg2.Error) as error:
            print(f"Error fetching symbols and dates: {error}")
        finally:
            connection.close()
           # print(symbols_dates_df)

symbols_dates_df = fetch_symbols_and_last_dates_AWS_postgreSQL()
print(symbols_dates_df)


      symbol    lastdate
0     XUEK.L  2024-04-25
1     0NOL.L  2024-04-22
2     0KVR.L  2024-04-25
3     ASIT.L  2024-04-24
4      JMG.L  2024-04-23
...      ...         ...
4523  MHPC.L  2024-04-23
4524   EUA.L  2024-04-24
4525  UC87.L  2024-04-23
4526  0HOB.L  2024-04-24
4527   ROR.L  2024-04-24

[4528 rows x 2 columns]


C:\Users\agd986\AppData\Local\Temp\ipykernel_2456\1627874347.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  symbols_dates_df = pd.read_sql(query, connection)


In [13]:
#*****************
#This function reads the data from the facts.dates_of_stock_info and compares the dates 
#Repeatable
#*****************

def fetch_new_stock_data_for_symbol_AWS_postgreSQL(symbol, start_date):
    end_date = datetime.now().strftime('%Y-%m-%d')
    stock = yf.Ticker(symbol)
    try:
        new_data = stock.history(start=start_date, end=end_date)
        if new_data.empty:
            # If no data is found for the symbol, print a message and return an empty DataFrame
            print(f"No data found for {symbol} between {start_date} and {end_date}.")
            return pd.DataFrame()
        else:
            new_data.reset_index(inplace=True)
            new_data['Symbol'] = symbol
            return new_data
    except IndexError as e:
        # Handle the specific 'index out of bounds' error
        print(f"Error fetching data for {symbol}: {e}. Returning empty DataFrame.")
        return pd.DataFrame()
    except Exception as e:
        # Handle any other unexpected errors
        print(f"Unexpected error fetching data for {symbol}: {e}. Returning empty DataFrame.")
        return pd.DataFrame()




In [14]:
#***************
#this function will insert data into historcal data table. 
#Repeatable
#Reference cell D
#Reference cell E
#SQL REFERENCE A
#***************


from datetime import datetime
def insert_new_stock_data_AWS_postgreSQL(df):
    insert_query = """
    INSERT INTO s_historical_stock_data (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Symbol)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    for index, row in df.iterrows():
        params = (
            row['Date'], row['Open'], row['High'], row['Low'], row['Close'],
            row['Volume'], row['Dividends'], row['Stock Splits'], row['Symbol']
        )
        execute_sql_query_AWS_postgreSQL(insert_query, params)
print(insert_new_stock_data_AWS_postgreSQL(pd.DataFrame()))

None


In [15]:
if __name__ == "__main__":
    symbols_dates_df = fetch_symbols_and_last_dates_AWS_postgreSQL()
    if not symbols_dates_df.empty:
        new_stock_data = pd.DataFrame()
        for index, row in symbols_dates_df.iterrows():
            symbol = row['symbol']
            last_date = row['lastdate'] + pd.Timedelta(days=1)  # Start from the day after the last recorded date
            symbol_new_data = fetch_new_stock_data_for_symbol_AWS_postgreSQL(symbol, last_date.strftime('%Y-%m-%d'))
            new_stock_data = pd.concat([new_stock_data, symbol_new_data], ignore_index=True)
        if not new_stock_data.empty:
            insert_new_stock_data_AWS_postgreSQL(new_stock_data)
        print("Stock data update process completed.")
    else:
        print("No symbols or last dates found.")

C:\Users\agd986\AppData\Local\Temp\ipykernel_2456\1627874347.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  symbols_dates_df = pd.read_sql(query, connection)
0NF3.L: No price data found, symbol may be delisted (1d 2024-01-23 -> 2024-04-29)


No data found for 0NF3.L between 2024-01-23 and 2024-04-29.


MATE.L: No price data found, symbol may be delisted (1d 2024-03-28 -> 2024-04-29)


No data found for MATE.L between 2024-03-28 and 2024-04-29.


0DRH.L: No price data found, symbol may be delisted (1d 2024-03-16 -> 2024-04-29)


...

0GP7.L: No price data found, symbol may be delisted (1d 2024-03-19 -> 2024-04-29)


No data found for 0GP7.L between 2024-03-19 and 2024-04-29.


0KAS.L: No price data found, symbol may be delisted (1d 2024-03-28 -> 2024-04-29)


No data found for 0KAS.L between 2024-03-28 and 2024-04-29.


KeyboardInterrupt: 

In [18]:
# Define a function to insert new financial data into the dimension.d_financial_info table.
def insert_into_financial_info_AWS_postgreSQL():
    """
    This function inserts new records into the dimension.d_financial_info table
    from the source.historical_stock_data table. It ensures uniqueness by not inserting
    any records that match existing symbol and date combinations in the target table,
    thereby preventing duplicate entries.
    """
    
    # SQL command for inserting data into dimension.d_financial_info table.
    # The INSERT INTO SELECT statement is used to select data from the source.historical_stock_data table
    # and insert it into the dimension.d_financial_info table.
    # The WHERE NOT EXISTS clause checks if a record with the same symbol and date already exists
    # in the dimension.d_financial_info table, preventing duplicate entries.
    insert_query = """
    INSERT INTO di_financial_info (
        symbol, 
        date,
        open,
        high,
        low,
        close,
        volume,
        dividends,
        stock_splits
    )
    SELECT 
        symbol, 
        date,
        open,
        high,
        low,
        close,
        volume,
        dividends,
        stock_splits
    FROM 
        s_historical_stock_data AS s
    WHERE NOT EXISTS (
        SELECT 1 
        FROM di_financial_info AS d 
        WHERE 
            d.symbol = s.symbol 
            AND d.date = s.date
    );
    """

    # Execute the SQL command using a try-except block.
    # This block attempts to run the insert_query and catches any exceptions that occur,
    # printing an error message if an exception is raised.
    try:
        execute_sql_query_AWS_postgreSQL(insert_query)
        # If the SQL command executes successfully, print a confirmation message.
        print("Data inserted into di_financial_inf successfully.")
    except Exception as e:
        # If an error occurs during the execution of the SQL command, print an error message.
        # The error message includes the exception details to help diagnose the issue.
        print(f"An error occurred while inserting data: {e}")


In [27]:
#Truncate the f_daily_facts
# SQL command to start a new database transaction.
# This ensures that all subsequent operations are part of a single transaction,
# which can be either committed or rolled back in case of an error.
begin_transaction = "BEGIN;"
# SQL command to clear the 'facts.daily_facts' table.
# This removes all existing records to prepare for inserting updated data.
# It's important to be cautious with this operation in a production environment.
truncate_table = "TRUNCATE f_daily_facts;"

execute_sql_query_AWS_postgreSQL(begin_transaction)
execute_sql_query_AWS_postgreSQL(truncate_table)


In [28]:
############
#Part of the Data refresh
#Define a function to update the 'f_daily_facts' table in the database.
#


#Reference cell H Name
def update_facts_table_AWS_postgreSQL():


    # SQL command to insert updated data into the 'facts.daily_facts' table.
    # This command performs a SELECT operation joining several tables to gather
    # the necessary data (like open, high, low, close prices, volume, and various growth metrics)
    # and inserts the resulting dataset into the 'facts.daily_facts' table.
    insert_data = """
    INSERT INTO public.f_daily_facts(
        Date, IsWeekend, symbol,  
        open, high, low, close, volume, 
        profit_margins, earnings_growth, revenue_growth 
    ) 
    SELECT 
        dd.date as date, 
        dd.isweekend as isweekend,
        df.symbol AS symbol,
        df.open AS open,
        df.high AS high,
        df.low AS low,
        df.close AS close,
        df.volume AS volume, 
        dc.profit_margins AS profit_margins,
        dc.earnings_growth AS earnings_growth,
        dc.revenue_growth AS revenue_growth
    FROM 
        di_financial_info as df 
    JOIN
        d_dated AS dd ON df.date = dd.Date
    JOIN
        d_customer AS dc ON df.symbol = dc.symbol;
    """



    # Attempt to execute the SQL commands within a try-except block to handle potential errors.
    try:
        # Execute the SQL commands one by one using a previously defined function.
        # This function manages the database connection and query execution.

        execute_sql_query_AWS_postgreSQL(insert_data)

        # Notify the user that the update process was successful.
        print("Facts table updated successfully.")
    except Exception as e:
        # If an error occurs during any of the SQL operations, print an error message.
        # The specific nature of the error is captured and displayed for troubleshooting.
        print(f"An error occurred while updating the facts table: {e}")


In [26]:
# SQL command to commit the transaction to the database.
# This finalizes the insertion of new data and ensures data integrity by
# making the changes permanent in the database.
commit_transaction = "COMMIT;"
execute_sql_query_AWS_postgreSQL(commit_transaction)

In [21]:



def update_dates_of_stock_info_AWS_postgreSQL():
    # SQL command to start a new transaction.
    # Beginning a transaction ensures that all the following operations are executed as a single unit of work,
    # allowing for rollback if something goes wrong, thus maintaining data consistency.
    begin_transaction = "BEGIN;"

    # SQL command to clear the 'facts.dates_of_stock_info' table.
    # This operation removes all existing records from the table, preparing it for the insertion of updated data.
    # Truncating a table is a drastic action and should be used with caution.
    truncate_table = "TRUNCATE f_dates_of_stock_info;"

    # SQL command to insert updated date information for each stock symbol into the 'facts.dates_of_stock_info' table.
    # This command aggregates data from the 'source.historical_stock_data' table, selecting the maximum date (i.e., the last date)
    # for each symbol and the current date, then groups the results by symbol.
    # The purpose is to keep a record of the last date for which data is available for each stock symbol,
    # as well as today's date to monitor the currency of the data.
    insert_data = """
    INSERT INTO f_dates_of_stock_info(symbol, lastdate, todays_date)
    SELECT symbol, MAX(date), CURRENT_DATE AS Todays_date FROM di_financial_info GROUP BY symbol;
    """

    # SQL command to commit the transaction to the database.
    # Committing the transaction makes all changes made during the transaction permanent,
    # ensuring that the data in the database is up to date and consistent.
    commit_transaction = "COMMIT;"

    # Attempt to execute the SQL commands within a try-except block to handle potential errors.
    try:
        # Execute the SQL commands sequentially using a predefined function to manage database connection and execution.
        # This function executes the SQL query against the database and commits or rolls back the transaction as necessary.
        execute_sql_query_AWS_postgreSQL(begin_transaction)
        execute_sql_query_AWS_postgreSQL(truncate_table)
        execute_sql_query_AWS_postgreSQL(insert_data)
        execute_sql_query_AWS_postgreSQL(commit_transaction)
        # Notify the user that the update process was successfully completed.
        print("Dates of stock info table updated successfully.")
    except Exception as e:
        # If an error occurs during the execution of the SQL commands, print an error message.
        # The message includes the specific error encountered to assist with troubleshooting.
        print(f"An error occurred while updating the dates of stock info table: {e}")
